In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from zeep import Client
from utils import safe_serialize, listar_ops

In [2]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [8]:
listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")

retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)
retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)
retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)
retornarProyectosAcuerdoXAnno(prmAnno)
retornarProyectoAcuerdo(prmProyectoAcuerdoId)


In [11]:
df_periodos = pd.read_csv(DATA_DIR / 'periodos.csv')
df_periodos

,Id,Periodo
0,1,1990-1994
1,2,1994-1998
2,3,1998-2002
3,4,2002-2006
4,5,2006-2010
5,6,2010-2014
6,7,1965-1969
7,8,2014-2018
8,9,2018-2022
9,10,2022-2026


In [34]:
def retornarProyectosAcuerdoXAnno(prmAnno):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosAcuerdo.asmx?WSDL")
    res = c.service.retornarProyectosAcuerdoXAnno(prmAnno)
    d = safe_serialize(res) or {}
    df = pd.json_normalize(d, sep=".").drop(
    columns=['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio',
           'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados']).rename(columns={
    "Id": "id_proyecto",
    "Numero": "numero_boletin",
    "Materia": "materia",
    "FechaIngreso": "fecha_ingreso",
    "Estado._value_1": "estado_nombre",
    "Estado.Valor": "estado_codigo",
    "Periodo.Id": "id_periodo",
    "Periodo.Nombre": "periodo_nombre"
})
    
    return df

def retornarProyectosResolucionXAnno(prmAnno):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSProyectosResolucion.asmx?WSDL")
    res = c.service.retornarProyectosResolucionXAnno(prmAnno)
    d = safe_serialize(res) or {}
    df = pd.json_normalize(d, sep=".").drop(columns=['Votacion', 'Autores',
        'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio',
        'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados']).rename(columns={
    "Id": "id_proyecto",
    "Numero": "numero_resolucion",
    "FechaIngreso": "fecha_ingreso",
    "Materia": "materia",
    "Estado._value_1": "estado_codigo",
    "Estado.Valor": "estado_nombre",
    "Periodo.Id": "id_periodo",
    "Periodo.Nombre": "periodo_nombre"
})
    return df

In [49]:
for periodo in df_periodos["Periodo"]:
    print(f"\nProcesando período {periodo}...")
    anio_inicio, anio_fin = map(int, periodo.split('-'))
    all_years = range(anio_inicio, anio_fin + 1)

    dfs_acuerdo, dfs_resolucion = [], []

    for anio in all_years:
        print(f"Año {anio}...")

        # === Proyectos de Acuerdo ===
        try:
            df_a = retornarProyectosAcuerdoXAnno(anio)
            if not df_a.empty:
                dfs_acuerdo.append(df_a)
                print(f"Acuerdos: {len(df_a)} proyectos")
            else:
                print(f"Acuerdos: sin datos")
        except Exception as e:
            print(f"Error acuerdos {anio}: {e}")

        # === Proyectos de Resolución ===
        try:
            df_r = retornarProyectosResolucionXAnno(anio)
            if not df_r.empty:
                dfs_resolucion.append(df_r)
                print(f"Resoluciones: {len(df_r)} proyectos")
            else:
                print(f"Resoluciones: sin datos")
        except Exception as e:
            print(f"Error resoluciones {anio}: {e}")

    # === Guardar por período ===
    output_dir = DATA_DIR / periodo
    output_dir.mkdir(parents=True, exist_ok=True)

    if dfs_acuerdo:
        df_out_a = pd.concat(dfs_acuerdo, ignore_index=True)
        output_a = output_dir / "ProyectosAcuerdo.csv"
        df_out_a.to_csv(output_a, index=False, encoding="utf-8")
        print(f"Guardado: {output_a} ({len(df_out_a)} filas)")
    else:
        print(f"Sin datos de acuerdos para {periodo}")

    if dfs_resolucion:
        df_out_r = pd.concat(dfs_resolucion, ignore_index=True)
        output_r = output_dir / "ProyectosResolucion.csv"
        df_out_r.to_csv(output_r, index=False, encoding="utf-8")
        print(f"Guardado: {output_r} ({len(df_out_r)} filas)")
    else:
        print(f"Sin datos de resoluciones para {periodo}")

print("\nProceso completado con éxito.")


Procesando período 1990-1994...
Año 1990...
Error acuerdos 1990: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Error resoluciones 1990: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Año 1991...
Error acuerdos 1991: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Error resoluciones 1991: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Año 1992...
Error acuerdos 1992: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTe

Error acuerdos 2006: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Error resoluciones 2006: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Sin datos de acuerdos para 2002-2006
Sin datos de resoluciones para 2002-2006

Procesando período 2006-2010...
Año 2006...
Error acuerdos 2006: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Error resoluciones 2006: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Año 2007...
Error acuerdos 2007: "['Votacion', 'Autores', 'OficiosEnviado

Acuerdos: 17 proyectos
Resoluciones: 547 proyectos
Guardado: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\2018-2022\ProyectosAcuerdo.csv (46 filas)
Guardado: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\2018-2022\ProyectosResolucion.csv (1987 filas)

Procesando período 2022-2026...
Año 2022...
Acuerdos: 17 proyectos
Resoluciones: 547 proyectos
Año 2023...
Acuerdos: 1 proyectos
Resoluciones: 596 proyectos
Año 2024...
Error acuerdos 2024: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Resoluciones: 278 proyectos
Año 2025...
Error acuerdos 2025: "['Votacion', 'Autores', 'OficiosEnviados', 'RespuestasRecibidas', 'Periodo.FechaInicio', 'Periodo.FechaTermino', 'Periodo.Legislaturas', 'Periodo.Diputados'] not found in axis"
Resoluciones: 236 pr